# Загрузка документов и сохранение эмбеддингов в базу

In [10]:
import yaml
import arxiv
import os
from langchain_community.document_loaders import PyPDFLoader
import chromadb
from tqdm import tqdm
import logging
from sentence_transformers import SentenceTransformer

C:\Users\Igorexy\AppData\Roaming\Python\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [1]:
# === Считывание тем из YAML ===
def load_topics_from_yaml(file_path: str):
    """Считывает YAML файл и возвращает список тем."""
    with open(file_path, 'r') as file:
        data = yaml.safe_load(file)
    return data['topics']

In [2]:
# === Загрузка статей с arXiv ===
def download_arxiv_papers(query: str, max_results: int, download_folder: str):
    """Загружает статьи с arXiv по запросу."""
    if not os.path.exists(download_folder):
        os.makedirs(download_folder)

    search = arxiv.Search(
        query=query,
        max_results=max_results,
        # sort_by=arxiv.SortCriterion.SubmittedDate
    )

    for result in search.results():
        paper_id = result.entry_id.split('/')[-1]
        pdf_path = os.path.join(download_folder, f'{paper_id}.pdf')
        if not os.path.exists(pdf_path):
            print(f"Downloading {result.title}...")
            result.download_pdf(download_folder, f'{paper_id}.pdf')
            print(f"Saved to {pdf_path}")
        else:
            print(f"{result.title} already downloaded.")

In [3]:
# === Считывание текста из PDF ===
def extract_text_from_pdf(file_path: str, extract_images=True):
    """Извлекает текст из PDF файла."""
    loader = PyPDFLoader(file_path, extract_images=extract_images)
    pages = loader.load()
    document_text = ' '.join([page.page_content for page in pages])
    return document_text, pages

In [4]:
import pdfplumber

def extract_text_from_pdf_columns(pdf_path):
    text = []
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            # text += page.extract_text(x_tolerance=1, y_tolerance=1, layout=True) + "\n"  # Коррекция разброса колонок
            text += page.extract_words()	
    return text

res = extract_text_from_pdf_columns(r"E:\ImportantFiles\Documents\University\Magic App\dataset_short\Machine Learning\clustering\1004.0694v1.pdf")
print(res[1])

{'text': 'rpA', 'x0': 16.34, 'x1': 36.34, 'top': 224.15999999999997, 'doctop': 224.15999999999997, 'bottom': 255.26, 'upright': False, 'height': 31.100000000000023, 'width': 20.000000000000004, 'direction': 'ttb'}


In [7]:
text.count('\f')

30

In [8]:
# Новая функция в надежде избавиться от лишних символов
import fitz  # PyMuPDF

# def extract_text_from_pdf_fitz(pdf_path):
#     doc = fitz.open(pdf_path)
#     text = ""
#     for page in doc:
#         text += page.get_text()

#     # Удаление служебных символов: переносы строк, лишние пробелы
#     clean_text = ' '.join(text.split())
#     return clean_text

def extract_text_from_pdf_fitz(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        # Извлекаем только текст, игнорируя графику и другие объекты
        text += page.get_text("text")

    # Удаление служебных символов
    clean_text = ' '.join(text.split())
    return clean_text

In [9]:
model = SentenceTransformer("all-MiniLM-L6-v2")

def embed_texts(texts):
    return model.encode(texts).tolist()

NameError: name 'SentenceTransformer' is not defined

In [ ]:
# === Загрузка данных в ChromaDB ===
def upload_to_chromadb(
    documents: list,
    collection_name: str,
    db_path: str = "./chroma_storage",
    embedding_function=None  # Функция эмбеддингов передается параметром
):
    """Добавляет документы в коллекцию ChromaDB с поддержкой кастомных эмбеддингов."""
    client = chromadb.PersistentClient(path=db_path)
    
    # Создаем коллекцию с функцией эмбеддингов, если она передана
    if embedding_function:
        collection = client.get_or_create_collection(
            name=collection_name,
            embedding_function=embedding_function
        )
    else:
        collection = client.get_or_create_collection(name=collection_name)

    for doc in documents:
        data = {
            "ids": [doc["ids"]],
            "documents": [doc["documents"]],
            "metadatas": [doc["metadata"]],
        }
        # Если задана функция эмбеддингов, вычисляем и добавляем вектора
        if embedding_function:
            data["embeddings"] = [embedding_function([doc["documents"]])[0]]

        collection.add(**data)
    
    print(f"Uploaded {len(documents)} documents to collection '{collection_name}'.")

In [ ]:
# Настройка логирования
logging.basicConfig(
    filename="log/error_arxiv.txt",
    level=logging.ERROR,
    format="%(asctime)s - %(levelname)s - %(message)s"
)


# Запуск

In [11]:
# Шаг 1. Считываем темы из YAML
topics = load_topics_from_yaml('config/topics_short.yaml')
print(f"Loaded topics: {[topic['name'] for topic in topics]}")

Loaded topics: ['Machine Learning', 'Data Analysis', 'Optimization Techniques', 'Natural Language Processing', 'Computer Vision', 'Theoretical Foundations', 'Applied AI']


In [ ]:
# Шаг 2. Загрузка статей по каждой теме
max_results = 50


for topic in tqdm(topics[6:]):
    topic_name = topic['name']
    folder = f"dataset/{topic_name}"#.replace(' ', '_')}"

    for keyword in topic['keywords']:
        query = f"all:\"{keyword}\""
        try:
            download_arxiv_papers(query, max_results=max_results, download_folder=folder + f'/{keyword}')
        except Exception as e:
            # Логирование ошибки
            logging.error(f"Ошибка при загрузке статей для темы '{topic_name}', ключевого слова '{keyword}': {e}")
            # (Необязательно) Вывод сообщения об ошибке в консоль
            print(f"Ошибка: {e} - при обработке темы '{topic_name}', ключевого слова '{keyword}'")

#### Модифицированное извлечение текста

In [ ]:
# Шаг 1. Считываем темы из YAML
topics = load_topics_from_yaml('config/topics.yaml')
print(f"Loaded topics: {[topic['name'] for topic in topics]}")

Loaded topics: ['Machine Learning', 'Data Analysis', 'Optimization Techniques', 'Natural Language Processing', 'Computer Vision', 'Theoretical Foundations', 'Applied AI', 'Emerging Topics']


In [23]:
# === Считывание текста из PDF ===
import re

def mode_extract_text_from_pdf(file_path: str):
    """Извлекает текст из PDF файла."""

    document_text = extract_text_from_pdf_fitz(file_path)

    cleaned_page = document_text

    # Удаляем спецсимволы, оставляя буквы, цифры, пробелы и пунктуацию
    cleaned_page = re.sub(r'[^а-яА-ЯёЁa-zA-Z0-9\s.,!?-]', '', cleaned_page)

    # Убираем лишние переносы строк
    cleaned_page = re.sub(r'\n{2,}', '\n', cleaned_page)

    # Удаляем ссылки на литературу вида [1], [2], ...
    cleaned_page = re.sub(r'\[\d+\]', '', cleaned_page)

    # Удаляем email-адреса
    cleaned_page = re.sub(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b', '', cleaned_page)

    # Удаляем артикулы на arXiv
    cleaned_page = re.sub(r'arXiv:\d+\.\d+(v\d+)? \[\w+(\.\w+)?\] \d+ \w+ \d+', '', cleaned_page)

    # Удаляем формулы, заключенные в $...$ или \( ... \)
    cleaned_page = re.sub(r'\$.*?\$|\\\(.*?\\\)', '', cleaned_page)

    # Удаляем числа
    cleaned_page = re.sub(r'\d+', '', cleaned_page)

    # Удаляем математические символы и спецсимволы
    cleaned_page = re.sub(r'[+\-*/=<>^~_|&%$#@!]', '', cleaned_page)

    # Удаляем гиперссылки
    cleaned_page = re.sub(r'http\S+', '', cleaned_page)

    # Убираем лишние пробелы
    cleaned_page = re.sub(r'\s+', ' ', cleaned_page).strip()

    # Проверяем наличие слова REFERENCES (регистр не учитывается)
    if re.search(r'\bREFERENCES\b', cleaned_page, flags=re.IGNORECASE):
        # Удаляем текст после "REFERENCES" (регистр не имеет значения)
        cleaned_page = re.split(r'\bREFERENCES\b', cleaned_page, flags=re.IGNORECASE)[0]

    cleaned_text = cleaned_page

    return cleaned_text


In [ ]:
from pdfminer.high_level import extract_text
# text = extract_text(r"E:\ImportantFiles\Documents\University\Magic App\dataset_short\Machine Learning\clustering\1808.08317v1.pdf")
text = extract_text(r"E:\ImportantFiles\Documents\University\Magic App\dataset_short\Data Analysis\data visualization\1705.01483v1.pdf")

In [ ]:
# Этот метод не использовался в этом файле
import re


def replace_ligatures(text):
    ligatures = {
    "ﬀ": "ff", "ﬁ": "fi", "ﬂ": "fl", "ﬃ": "ffi", "ﬄ": "ffl", "ﬅ": "ft", "ﬆ": "st",
    "Æ": "AE", "Œ": "OE", "Ǆ": "DZ", "ǅ": "Dz",
    "Ϝ": "W", "Ϟ": "KS",
    "Ꜳ": "AA", "ꜳ": "aa", "Ꜵ": "AO"
}

    pattern = re.compile("|".join(re.escape(k) for k in ligatures))
    return pattern.sub(lambda m: ligatures[m.group()], text)

def clean_text(text):
    text = replace_ligatures(text)  # Удаляем лигатуры
    text = re.sub(r'\f', '', text)  # Удаляем символы \f
    text = re.sub(r'(?m)^.$', '', text)  # Удаляем строки с одним символом
    text = re.sub(r'(?<![.!?])\n(?!\n)', ' ', text)  # Убираем лишние переносы строк
    text = re.sub(r'(?<=\w)-\n', '', text)  # Убираем переносы слов
    text = re.sub(r'\n{2,}', '\n', text)  # Сводим подряд идущие переносы строк к одному
    text = re.sub(r'\bREFERENCES\b.*', '', text, flags=re.IGNORECASE | re.DOTALL)  # Удаляем все после REFERENCES
    text = re.sub(r'\d{4,}.*', '', text)  # Удаляем непонятные числовые строки
    text = re.sub(r'(?m)^\s*\d+\.?\s*$', '', text)  # Удаляем строки с номерами
    text = re.sub(r'(?m)^([A-Za-z]+\s*){1,3}\d+$', '', text)  # Удаляем табличные данные
    return text.strip()

# Пример использования:
raw_text = """Clustering is an ubiquitous data analysis tool applied across diverse disciplines, such as\n
bioinformatics, marketing, and image segmentation. Its wide utility is perhaps unsurprising,\n
as its intuitive aim - to divide data into groups of similar items - applies at various stages of\n
modality tests, before delving into clusterability methods.\n
REFERENCES\n1234 Some reference text that should be removed."""

cleaned_text = clean_text(text)
print(cleaned_text)

#### извлечение по документам, без учета страниц

In [24]:
# Шаг 3. Извлечение текста из PDF

all_documents = []

for topic in tqdm([topics[0]]):
# for topic in tqdm(topics[5:]):
    topic_name = topic['name']
    folder = rf"dataset_short\{topic_name}"

    for keyword in os.listdir(folder):
        folder_keywords = os.path.join(folder, keyword)
        print(folder_keywords)

        for file_name in os.listdir(folder_keywords):
            if file_name.endswith('.pdf'):
                file_path = os.path.join(folder_keywords, file_name)

                try:
                    # Предполагается, что функция extract_text_from_pdf возвращает общий текст и список страниц
                    document_text = mode_extract_text_from_pdf(file_path)
                except Exception as e:
                    logging.error(f"Ошибка при считывании текста для темы '{topic_name}', ключевого слова '{keyword}': {e}")
                    print(f"Ошибка: {e} - при обработке темы '{topic_name}', ключевого слова '{keyword}'")
                    continue

                # Формируем записи для документа
                all_documents.append({
                    "ids": file_name.split('.pdf')[0],
                    "documents": document_text,
                    "metadata": {
                        "topic": topic_name,
                        "keyword": keyword,  # Добавляем ключевое слово в метаданные
                        "filename": file_name,
                    }
                })

print(f"Extracted text from {len(all_documents)} documents.")

  0%|          | 0/1 [00:00<?, ?it/s]

dataset_short\Machine Learning\clustering
dataset_short\Machine Learning\decision trees
dataset_short\Machine Learning\deep learning
dataset_short\Machine Learning\ensemble methods
dataset_short\Machine Learning\neural networks
dataset_short\Machine Learning\reinforcement learning
MuPDF error: syntax error: could not parse color space (313 0 R)

MuPDF error: syntax error: could not parse color space (432 0 R)

MuPDF error: syntax error: could not parse color space (550 0 R)

MuPDF error: syntax error: could not parse color space (659 0 R)

MuPDF error: syntax error: could not parse color space (771 0 R)

MuPDF error: syntax error: could not parse color space (993 0 R)

MuPDF error: syntax error: could not parse color space (1289 0 R)

MuPDF error: syntax error: could not parse color space (1326 0 R)

MuPDF error: syntax error: could not parse color space (1601 0 R)

dataset_short\Machine Learning\supervised learning
dataset_short\Machine Learning\SVM
dataset_short\Machine Learning\unsu

100%|██████████| 1/1 [00:10<00:00, 10.10s/it]

Extracted text from 180 documents.


В прошлый раз ушло 2 минуты на считывание только Machine Learning

In [ ]:
# Шаг 4. Загрузка документов в ChromaDB
upload_to_chromadb(all_documents, collection_name="magic_document", embedding_function=embed_texts)

KeyboardInterrupt: 